In [47]:
import pandas as pd

df = pd.read_csv('train.tsv', sep='\t')
df_test = pd.read_csv('test.tsv', sep='\t')

In [48]:
df.shape

(156060, 4)

In [49]:
df = df.sample(frac=0.3)
df.shape

(46818, 4)

In [50]:
import numpy as np

seq_len = 256
num_samples = len(df)

Xids = np.zeros((num_samples, seq_len))
Xmask = np.zeros((num_samples, seq_len))

Xids.shape

(46818, 256)

In [51]:
!pip install -q transformers

In [52]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

for i, phrase in enumerate(df['Phrase']):
  tokens = tokenizer.encode_plus(phrase, max_length=seq_len, truncation=True,
                                 padding='max_length', add_special_tokens=True,
                                 return_tensors='tf')

  Xids[i, :] = tokens['input_ids']
  Xmask[i, :] = tokens['attention_mask']

In [53]:
arr = df['Sentiment'].values

In [54]:
labels = np.zeros((num_samples, arr.max()+1))

In [55]:
labels.shape

(46818, 5)

In [56]:
labels[np.arange(num_samples), arr] = 1

In [57]:
import tensorflow as tf

In [58]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))
dataset

<_TensorSliceDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [59]:
def map_func(input_ids, masks, labels):
  return {'input_ids' : input_ids, 'attention_mask' : masks}, labels

In [60]:
dataset = dataset.map(map_func)
dataset

<_MapDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [61]:
batch_size=16

dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)
dataset

<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [62]:
split = 0.9
size = int((num_samples / batch_size) * split)

In [63]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

**Model**

In [64]:
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained('bert-base-uncased')

bert.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "tf_bert_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
Total params: 109,482,240
Trainable params: 109,482,240
Non-trainable params: 0
_________________________________________________________________


In [65]:
input_ids = tf.keras.layers.Input(shape=(seq_len,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(seq_len,), name='attention_mask', dtype='int32')

embeddings = bert.bert(input_ids, attention_mask=mask)[1]

x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
y = tf.keras.layers.Dense(arr.max()+1, activation='softmax', name='outputs')(x)

In [66]:
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

model.layers[2].trainable=False
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

In [67]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, weight_decay=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [68]:
model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=1)

  97/2633 [>.............................] - ETA: 14:43:32 - loss: 1.3375 - accuracy: 0.4910